In [47]:


import tensorflow as tf
print(tf.__version__)

2.12.0


In [48]:
import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import Model, Sequential
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError

from tensorflow.keras.layers import Dense, Conv1D, LSTM, Lambda, Reshape, RNN, LSTMCell

import warnings
warnings.filterwarnings('ignore')

In [49]:
plt.rcParams['figure.figsize'] = (10, 7.5)
plt.rcParams['axes.grid'] = False

In [50]:
tf.random.set_seed(42)
np.random.seed(42)

In [51]:
train_df = pd.read_csv('data/lome_train1.csv', index_col='date_time',parse_dates=True)
val_df = pd.read_csv('data/lome_val1.csv', index_col='date_time',parse_dates=True)
test_df = pd.read_csv('data/lome_test1.csv', index_col='date_time',parse_dates=True)

print(train_df.shape, val_df.shape, test_df.shape)


(3735, 21) (1068, 21) (534, 21)


In [52]:
#DataWindow Class

In [53]:
class DataWindow():
    def __init__(self, input_width, label_width, shift,
                 train_df=train_df, val_df=val_df, test_df=test_df,
                 label_columns=None):

        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
        self.column_indices = {name: i for i, name in enumerate(train_df.columns)}

        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def split_to_inputs_labels(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:,:,self.column_indices[name]] for name in self.label_columns],
                axis=-1
            )
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels

    def plot(self, model=None, plot_col='rain_sum (mm)', max_subplots=3):
        inputs, labels = self.sample_batch

        plt.figure(figsize=(12, 8))
        plot_col_index = self.column_indices[plot_col]
        max_n = min(max_subplots, len(inputs))

        for n in range(max_n):
            plt.subplot(3, 1, n+1)
            plt.ylabel(f'{plot_col} [scaled]')
            plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                     label='Inputs', marker='.', zorder=-10)

            if self.label_columns:
              label_col_index = self.label_columns_indices.get(plot_col, None)
            else:
              label_col_index = plot_col_index

            if label_col_index is None:
              continue

            plt.scatter(self.label_indices, labels[n, :, label_col_index],
                        edgecolors='k', marker='s', label='Labels', c='green', s=64)
            if model is not None:
              predictions = model(inputs)
              plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                          marker='X', edgecolors='k', label='Predictions',
                          c='red', s=64)

            if n == 0:
              plt.legend()

        plt.xlabel('Time (h)')

    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32
        )

        ds = ds.map(self.split_to_inputs_labels)
        return ds

    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

    @property
    def sample_batch(self):
        result = getattr(self, '_sample_batch', None)
        if result is None:
            result = next(iter(self.train))
            self._sample_batch = result
        return result

In [54]:
class Baseline(Model):
    def __init__(self, label_index=None):
        super().__init__()
        self.label_index = label_index

    def call(self, inputs):
        if self.label_index is None:
            return inputs

        elif isinstance(self.label_index, list):
            tensors = []
            for index in self.label_index:
                result = inputs[:, :, index]
                result = result[:, :, tf.newaxis]
                tensors.append(result)
            return tf.concat(tensors, axis=-1)

        result = inputs[:, :, self.label_index]
        return result[:,:,tf.newaxis]

Multi-output baseline model


In [55]:
mo_single_step_window = DataWindow(input_width=1, label_width=1, shift=1, label_columns=['precipitation_sum (mm)','rain_sum (mm)','river_discharge','intensity_rain','intensity_flood','intensity_drought'])
mo_wide_window = DataWindow(input_width=14, label_width=14, shift=14, label_columns=['precipitation_sum (mm)','rain_sum (mm)','river_discharge','intensity_rain','intensity_flood','intensity_drought'])

In [56]:
column_indices = {name: i for i, name in enumerate(train_df.columns)}

In [57]:
print(column_indices['precipitation_sum (mm)'])
print(column_indices['rain_sum (mm)'])
print(column_indices['river_discharge'])
print(column_indices['intensity_rain'])
print(column_indices['intensity_flood'])
print(column_indices['intensity_drought'])

7
8
17
18
19
20


In [58]:
mo_baseline_last = Baseline(label_index=[7,8,17,18,19,20])

mo_baseline_last.compile(loss=MeanSquaredError(), metrics=[MeanAbsoluteError()])

#mo_val_performance = {}
#mo_performance = {}

#mo_val_performance['Baseline - Last'] = mo_baseline_last.evaluate(mo_wide_window.val)
#mo_performance['Baseline - Last'] = mo_baseline_last.evaluate(mo_wide_window.test, verbose=0)

Implementing a deep neural network as a multi-output model

In [59]:
def compile_and_fit(model, window, patience=3, max_epochs=50):
    early_stopping = EarlyStopping(monitor='val_loss',
                                   patience=patience,
                                   mode='min')

    model.compile(loss=MeanSquaredError(),
                  optimizer=Adam(),
                  metrics=[MeanAbsoluteError()])

    history = model.fit(window.train,
                       epochs=max_epochs,
                       validation_data=window.val,
                       callbacks=[early_stopping])

    return history

In [60]:
mo_dense = Sequential([
    Dense(units=64, activation='relu'),
    Dense(units=64, activation='relu'),
    Dense(units=6,activation='relu')
])

history = compile_and_fit(mo_dense, mo_wide_window)

#mo_val_performance['Dense'] = mo_dense.evaluate(mo_single_step_window.val)
#mo_performance['Dense'] = mo_dense.evaluate(mo_single_step_window.test, verbose=0)

Epoch 1/50


2023-09-24 11:51:45.646192: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [3735,21]
	 [[{{node Placeholder/_0}}]]
2023-09-24 11:51:45.646786: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [3735,21]
	 [[{{node Placeholder/_0}}]]


 93/116 [=======================>......] - ETA: 0s - loss: 12.3126 - mean_absolute_error: 1.1264

2023-09-24 11:51:46.649299: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [1068,21]
	 [[{{node Placeholder/_0}}]]
2023-09-24 11:51:46.649878: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype int32 and shape [1041]
	 [[{{node Placeholder/_8}}]]


116/116 [==============================] - 1s 4ms/step - loss: 12.6004 - mean_absolute_error: 1.1042 - val_loss: 17.6063 - val_mean_absolute_error: 1.1113
Epoch 2/50
116/116 [==============================] - 0s 3ms/step - loss: 11.1131 - mean_absolute_error: 1.0005 - val_loss: 17.4091 - val_mean_absolute_error: 1.1571
Epoch 3/50
116/116 [==============================] - 0s 3ms/step - loss: 11.0178 - mean_absolute_error: 0.9936 - val_loss: 17.8241 - val_mean_absolute_error: 1.0423
Epoch 4/50
116/116 [==============================] - 0s 3ms/step - loss: 11.0433 - mean_absolute_error: 0.9937 - val_loss: 17.4146 - val_mean_absolute_error: 1.1556
Epoch 5/50
116/116 [==============================] - 0s 3ms/step - loss: 10.9835 - mean_absolute_error: 0.9896 - val_loss: 17.5154 - val_mean_absolute_error: 1.3005


**LSTM**

In [61]:
mo_lstm_model = Sequential([
    LSTM(32, return_sequences=True),
    Dense(units = 6,activation='relu')
])

history = compile_and_fit(mo_lstm_model, mo_wide_window)

#mo_val_performance = {}
#mo_performance = {}

#mo_val_performance['LSTM'] = mo_lstm_model.evaluate(mo_wide_window.val)
#mo_performance['LSTM'] = mo_lstm_model.evaluate(mo_wide_window.test, verbose=0)

Epoch 1/50


2023-09-24 11:51:58.066902: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_13' with dtype int32
	 [[{{node Placeholder/_13}}]]
2023-09-24 11:51:58.067509: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype int32 and shape [3708]
	 [[{{node Placeholder/_8}}]]
2023-09-24 11:51:58.301589: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32


108/116 [==========================>...] - ETA: 0s - loss: 12.7628 - mean_absolute_error: 0.9925

2023-09-24 11:52:00.713338: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype int32 and shape [1041]
	 [[{{node Placeholder/_8}}]]
2023-09-24 11:52:00.714148: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype int32 and shape [1041]
	 [[{{node Placeholder/_8}}]]
2023-09-24 11:52:00.947514: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' wi

116/116 [==============================] - 3s 11ms/step - loss: 12.5873 - mean_absolute_error: 0.9872 - val_loss: 18.5922 - val_mean_absolute_error: 1.0608
Epoch 2/50
116/116 [==============================] - 1s 6ms/step - loss: 11.2254 - mean_absolute_error: 0.9612 - val_loss: 17.9371 - val_mean_absolute_error: 1.0935
Epoch 3/50
116/116 [==============================] - 1s 6ms/step - loss: 10.9906 - mean_absolute_error: 0.9775 - val_loss: 17.6394 - val_mean_absolute_error: 1.1158
Epoch 4/50
116/116 [==============================] - 1s 7ms/step - loss: 10.8777 - mean_absolute_error: 0.9715 - val_loss: 17.4626 - val_mean_absolute_error: 1.1367
Epoch 5/50
116/116 [==============================] - 1s 7ms/step - loss: 10.8248 - mean_absolute_error: 0.9693 - val_loss: 17.3824 - val_mean_absolute_error: 1.1204
Epoch 6/50
116/116 [==============================] - 1s 7ms/step - loss: 10.8094 - mean_absolute_error: 0.9705 - val_loss: 17.3237 - val_mean_absolute_error: 1.1035
Epoch 7/50
116

In [63]:
mo_lstm_model.save("lstm_model.h5")

In [64]:
new_model = keras.models.load_model("lstm_model.h5")

2023-09-24 11:52:24.482795: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-24 11:52:24.484603: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-24 11:52:24.486002: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [65]:
predicted_results = new_model.predict(mo_wide_window.test)
predicted_array= predicted_results[0]

my_array = np.array(predicted_array)

df = pd.DataFrame(my_array)


df2 = df.rename(columns={0: "precipitation_sum (mm)", 1: "rain_sum (mm)",2:"river_discharge",3:"intensity_rain",4:"intensity_flood",5:"intensity_drought"})

df2.head(14)

2023-09-24 11:52:27.980763: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype int32
	 [[{{node Placeholder/_14}}]]
2023-09-24 11:52:27.981354: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype int32 and shape [507]
	 [[{{node Placeholder/_8}}]]


 1/16 [>.............................] - ETA: 6s

2023-09-24 11:52:28.186406: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-24 11:52:28.187920: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-24 11:52:28.189148: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

16/16 [==============================] - 0s 2ms/step


,precipitation_sum (mm),rain_sum (mm),river_discharge,intensity_rain,intensity_flood,intensity_drought
0,2.217372,2.176384,0.0,0.005089,0.0,0.0
1,2.169400,1.969851,0.0,0.000000,0.0,0.0
2,2.829458,2.930498,0.0,0.000000,0.0,0.0
3,2.727690,2.828001,0.0,0.000000,0.0,0.0
4,2.880639,2.980734,0.0,0.038753,0.0,0.0
5,2.319134,2.257364,0.0,0.007510,0.0,0.0
6,2.906512,3.120770,0.0,0.121421,0.0,0.0
7,3.838314,4.034000,0.0,0.133911,0.0,0.0
8,3.966962,4.150835,0.0,0.134103,0.0,0.0
9,3.383754,3.434811,0.0,0.094804,0.0,0.0
